# Import statements & utilities

In [1]:
!pip install -q transformers[torch]

In [2]:
!pip install -q datasets

In [3]:
!pip -q install wandb

In [4]:
import wandb

In [5]:
from urllib import request
import pandas as pd
from collections import Counter
import random
import numpy as np
import pandas as pd
import copy
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import torch
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

In [6]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [7]:
from dont_patronize_me import DontPatronizeMe

In [8]:
dpm = DontPatronizeMe('.', 'task4_test.tsv')

In [9]:
dpm.load_task1()
dpm.load_task2(return_one_hot=True)

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


In [10]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

# Load data

In [11]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)


In [12]:
data=dpm.train_task1_df
data

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [13]:
rows = [] # will contain par_id, label and text
for idx in range(len(trids)):
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })

trdf1 = pd.DataFrame(rows)
trdf1

,par_id,community,text,label
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1
1,4136,homeless,Durban 's homeless communities reconciliation ...,1
2,10352,poor-families,The next immediate problem that cropped up was...,1
3,8279,vulnerable,Far more important than the implications for t...,1
4,1164,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...,...
8370,8380,refugee,Rescue teams search for survivors on the rubbl...,0
8371,8381,hopeless,The launch of ' Happy Birthday ' took place la...,0
8372,8382,homeless,"The unrest has left at least 20,000 people dea...",0
8373,8383,hopeless,You have to see it from my perspective . I may...,0


In [14]:
trdf1.to_csv("trdf1.csv")

In [15]:
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })
tedf1 = pd.DataFrame(rows)
tedf1

,par_id,community,text,label
0,4046,hopeless,We also know that they can benefit by receivin...,1
1,1279,refugee,Pope Francis washed and kissed the feet of Mus...,1
2,8330,refugee,Many refugees do n't want to be resettled anyw...,1
3,4063,in-need,"""Budding chefs , like """" Fred """" , """" Winston ...",1
4,4089,homeless,"""In a 90-degree view of his constituency , one...",1
...,...,...,...,...
2089,10462,homeless,"The sad spectacle , which occurred on Saturday...",0
2090,10463,refugee,""""""" The Pakistani police came to our house and...",0
2091,10464,disabled,"""When Marie O'Donoghue went looking for a spec...",0
2092,10465,women,"""Sri Lankan norms and culture inhibit women fr...",0


In [16]:
rows_list = tedf1.values.tolist()

# Shuffle the list using random.shuffle
random.shuffle(rows_list)

# Create a new DataFrame from the shuffled list
tedf1 = pd.DataFrame(rows_list, columns=tedf1.columns)
tedf1

,par_id,community,text,label
0,8490,homeless,Talking about family : I met some homeless peo...,0
1,8998,hopeless,""""""" It 's a hopeless situation , """" said the a...",0
2,10236,women,In celebration of South Africa and to show off...,0
3,9380,immigrant,The simulation began with officers taking 11 i...,0
4,10060,vulnerable,Search <h> Singapore 's big developers vulnera...,0
...,...,...,...,...
2089,9591,migrant,"The organizers of the event , which attracted ...",0
2090,9707,women,"I 've met countless women with rich , dark , g...",0
2091,9269,in-need,Soccerladuma is back again with lies . Bucs ar...,0
2092,9385,homeless,Various other areas have been experiencing exc...,0


In [ ]:
tedf1.to_csv("tedf1.csv")

# Load preprocessed data

In [12]:
# if data already saved
trdf1 = pd.read_csv("trdf1.csv", index_col=0)
trdf1

,par_id,community,text,label
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1
1,4136,homeless,Durban 's homeless communities reconciliation ...,1
2,10352,poor-families,The next immediate problem that cropped up was...,1
3,8279,vulnerable,Far more important than the implications for t...,1
4,1164,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...,...
8370,8380,refugee,Rescue teams search for survivors on the rubbl...,0
8371,8381,hopeless,The launch of ' Happy Birthday ' took place la...,0
8372,8382,homeless,"The unrest has left at least 20,000 people dea...",0
8373,8383,hopeless,You have to see it from my perspective . I may...,0


In [13]:
tedf1 = pd.read_csv("tedf1.csv", index_col=0)
tedf1

,par_id,community,text,label
0,9830,hopeless,Since Mr. Cruz 's election to the Senate in 20...,0
1,9165,refugee,These are some of the dilemmas that befell Min...,0
2,9265,immigrant,"Lucy Lawless ( Salem , Spartacus ) as Ruby , a...",0
3,8457,refugee,The situation has some parallels to Myanmar 's...,0
4,10122,poor-families,"Five years down the lane , the schools whose l...",0
...,...,...,...,...
2089,9170,migrant,"This May 5 , 2018 , photo , released by the Ro...",0
2090,9264,vulnerable,"""Asim Qureshi , of Cageprisoners , says that h...",0
2091,9396,immigrant,Mr Ariel told ministry of housing staff and of...,0
2092,2953,poor-families,""""""" The worst thing was dealt to us , """" he sa...",1


# Load model

In [17]:
# Load model

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Create Datasets & DataLoaders

In [18]:
import datasets
from datasets import Dataset
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer

In [19]:
# Split the dataset into train and test sets
train_df, val_df = train_test_split(trdf1, test_size=0.2, random_state=42)

train_dataset =  Dataset.from_dict({"data": train_df["text"], "label": train_df["label"]}).with_format("torch")
val_dataset =  Dataset.from_dict({"data": val_df["text"], "label": val_df["label"]}).with_format("torch")
test_dataset =  Dataset.from_dict({"data": tedf1["text"], "label": tedf1["label"]}).with_format("torch")

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

6700
1675
2094


In [20]:
def tokenize_function(example):
    return tokenizer(example["data"], truncation=True)


tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/6700 [00:00<?, ? examples/s]

Map:   0%|          | 0/1675 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

# Train Model

In [22]:
from sklearn.metrics import accuracy_score, f1_score

# Define evaluation metrics function
def compute_metrics(p):
    predictions = p.predictions.argmax(axis=1)
    labels = p.label_ids

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='binary')


    # return {"accuracy": accuracy, "f1": f1, "predictions": list(predictions)}
    return {"accuracy": accuracy, "f1": f1}

In [23]:
batch_size = 16

training_args = TrainingArguments("train_logs", evaluation_strategy="epoch",\
                                  learning_rate=2e-6,
                                  num_train_epochs=10,
                                  lr_scheduler_type="cosine",
                                  optim = "adamw_torch",
                                  weight_decay=0,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  run_name="roberta-base-hate-best",  # name of the W&B run (optional)
                                  logging_steps=1,  # how often to log to W&B
                                  )


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [24]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.049900,0.261408,0.895522,0.064171
2,0.089600,0.242177,0.897910,0.442997
3,0.219400,0.233192,0.910448,0.509804
4,0.477400,0.260001,0.913433,0.487633
5,0.005300,0.274739,0.911045,0.544343
6,0.044400,0.289490,0.910448,0.525316
7,0.032500,0.295957,0.912836,0.525974
8,0.008900,0.313606,0.912239,0.494845
9,0.010000,0.314517,0.912836,0.525974
10,0.003200,0.315550,0.912836,0.525974


TrainOutput(global_step=4190, training_loss=0.1763567305195561, metrics={'train_runtime': 1664.9021, 'train_samples_per_second': 40.243, 'train_steps_per_second': 2.517, 'total_flos': 4433028671788080.0, 'train_loss': 0.1763567305195561, 'epoch': 10.0})

# Predicitons on val set

In [25]:
predictions = trainer.predict(tokenized_val_dataset).predictions
predictions = predictions.argmax(axis=1)
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [26]:
Counter(predictions)

Counter({0: 1534, 1: 141})

In [27]:
labels2file([[k] for k in predictions], 'val.txt')

In [28]:
trainer.predict(tokenized_val_dataset).metrics

{'test_loss': 0.3155500888824463,
 'test_accuracy': 0.9128358208955224,
 'test_f1': 0.525974025974026,
 'test_runtime': 10.2246,
 'test_samples_per_second': 163.821,
 'test_steps_per_second': 10.269}

In [29]:
results = trainer.evaluate(tokenized_val_dataset)

print(results)

{'eval_loss': 0.3155500888824463, 'eval_accuracy': 0.9128358208955224, 'eval_f1': 0.525974025974026, 'eval_runtime': 10.2219, 'eval_samples_per_second': 163.864, 'eval_steps_per_second': 10.272, 'epoch': 10.0}


# Predictions on dev set (our test set)

In [30]:
predictions = trainer.predict(tokenized_test_dataset)
print(predictions.metrics)

predictions = predictions.predictions.argmax(axis=1)
predictions

{'test_loss': 0.259332537651062, 'test_accuracy': 0.9288443170964661, 'test_f1': 0.6047745358090185, 'test_runtime': 13.0253, 'test_samples_per_second': 160.765, 'test_steps_per_second': 10.057}


array([0, 0, 0, ..., 0, 0, 0])

In [31]:
labels2file([[k] for k in predictions], 'dev.txt')

In [32]:
Counter(predictions)

Counter({0: 1916, 1: 178})

# Predictions on official test set

In [33]:
dpm.load_test()
test_df = dpm.test_set_df

In [34]:
test_df

,par_id,art_id,keyword,country,text
0,t_0,@@7258997,vulnerable,us,"In the meantime , conservatives are working to..."
1,t_1,@@16397324,women,pk,In most poor households with no education chil...
2,t_2,@@16257812,migrant,ca,The real question is not whether immigration i...
3,t_3,@@3509652,migrant,gb,"In total , the country 's immigrant population..."
4,t_4,@@477506,vulnerable,ca,"Members of the church , which is part of Ken C..."
...,...,...,...,...,...
3827,t_3893,@@20319448,migrant,jm,In a letter dated Thursday to European Commiss...
3828,t_3894,@@9990672,poor-families,au,They discovered that poor families with health...
3829,t_3895,@@37984,migrant,ca,"She married at 19 , to Milan ( Emil ) Badovina..."
3830,t_3896,@@9691377,immigrant,us,The United Kingdom is n't going to devolve int...


In [35]:
official_test_dataset =  Dataset.from_dict({"data": test_df["text"]}).with_format("torch")
tokenized_official_test_dataset = official_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3832 [00:00<?, ? examples/s]

In [36]:
predictions = trainer.predict(tokenized_official_test_dataset)
print(predictions.metrics)

predictions = predictions.predictions.argmax(axis=1)
predictions

{'test_runtime': 25.0573, 'test_samples_per_second': 152.93, 'test_steps_per_second': 9.578}


array([0, 0, 0, ..., 0, 0, 0])

In [37]:
labels2file([[k] for k in predictions], 'test.txt')